# Librerías

In [20]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from pmdarima import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from skforecast.ForecasterSarimax import ForecasterSarimax
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.feature_selection import RFECV
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection_sarimax import backtesting_sarimax
import warnings
warnings.filterwarnings('ignore')

# ENTRENAMIENTO DE MODELOS

## Carga de datos

In [4]:
df = pd.read_csv("datos_preprocesados.csv", parse_dates=['fecha'], index_col='fecha')
df = df.asfreq('D')
df = df.sort_index()

## División del conjunto de datos

In [5]:
# Divisiones de los datos en los conjuntos de entrenamiento y test
end_train = '2023-08-12'

data_train = df.loc[:end_train]
data_test  = df.loc[end_train:]

print(f"Dates train      : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Dates test       : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

Dates train      : 2019-01-01 00:00:00 --- 2023-08-12 00:00:00  (n=1685)
Dates test       : 2023-08-12 00:00:00 --- 2024-09-26 00:00:00  (n=412)


## Funciones para ARIMA y ARIMAX

In [41]:
# Función para entrenamiento del modelo ARIMA. No usa optimizador.

def arima(p, d, q, maxiter):
    forecaster = ForecasterSarimax( regressor=ARIMA(order=(p, d, q), seasonal_order=(0, 0, 0, 0), maxiter=maxiter))
    
    metric, predictions = backtesting_sarimax(
        forecaster         = forecaster,
        y                  = df['demanda'],
        steps              = 1,
        metric             = 'mean_absolute_error',
        initial_train_size = len(data_train),
        fixed_train_size   = False,       
        refit              = False,
        verbose            = False,
        show_progress      = False
    )
    return metric.iloc[0]

In [42]:
# Función para entrenamiento del modelo ARIMAX. No usa optimizador.

def arimax(exog_features, p, d, q, maxiter):
    forecaster = ForecasterSarimax( regressor=ARIMA(order=(p, d, q), seasonal_order=(0, 0, 0, 0), maxiter=maxiter))
    
    metric, predictions = backtesting_sarimax(
        forecaster         = forecaster,
        y                  = df['demanda'],
        exog               = df[exog_features],
        steps              = 1,
        metric             = 'mean_absolute_error',
        initial_train_size = len(data_train),
        fixed_train_size   = False,       
        refit              = False,
        verbose            = False,
        show_progress      = False
    )
    return metric.iloc[0]          

## Función para un modelo genérico con variables exógenas

In [43]:
def generic_model(modelo, df, lags, exog_features, search_space, n_iter):

    if modelo == 'decision_tree':
        forecaster = ForecasterAutoreg(regressor = DecisionTreeRegressor(random_state=15926), lags = lags)

    if modelo == 'random_forest':
        forecaster = ForecasterAutoreg(regressor = RandomForestRegressor(random_state=151226), lags = lags)

    if modelo == 'xgboost':
        forecaster = ForecasterAutoreg(regressor = XGBRegressor(random_state=15942), lags = lags)
    

    results_search, frozen_trial = bayesian_search_forecaster(
    forecaster         = forecaster,
    y                  = df['demanda'],
    exog               = df[exog_features],
    search_space       = search_space,
    steps              = 1,
    refit              = False,
    metric             = 'mean_absolute_error',
    initial_train_size = len(data_train),
    fixed_train_size   = False,
    n_trials           = n_iter,
    random_state       = 123,
    return_best        = True,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = False
    )
    
    best_params = results_search['params'].iat[0]

    metric, predictions = backtesting_forecaster(
    forecaster         = forecaster,
    y                  = df['demanda'],
    exog               = df[exog_features],
    steps              = 1,
    metric             = 'mean_absolute_error',
    initial_train_size = len(data_train),
    refit              = False,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = False
    )

    return predictions, best_params, metric.iloc[0]

## Resultados de modelos ARIMA y ARIMAX

In [49]:
exog_features = ['diasem', 'trim', 'festivo']
maxiter = 50

# Definir los rangos para p, d y q
range_values = range(1, 2) 

In [51]:
# Ignorar advertencias específicas de statsmodels
warnings.filterwarnings("ignore", category=UserWarning, message="Non-stationary starting autoregressive parameters found")
warnings.filterwarnings("ignore", category=UserWarning, message="Non-invertible starting MA parameters found")
warnings.filterwarnings("ignore", category=FutureWarning, message="Series.__getitem__ treating keys as positions is deprecated")


start_time = time.time()

# Crear una lista para almacenar los resultados
results = []

# Iterar sobre los valores de p, d, q
for p in range_values:
    for d in range_values:
        for q in range_values:
            # Llamar a la función arimax con los parámetros actuales
            metrica = arimax(exog_features, p, d, q, maxiter)
            
            # Si la métrica es un DataFrame o una Serie, extrae el valor escalar
            if isinstance(metrica, pd.Series):
                metrica_value = metrica.iloc[0]  # Extrae el primer valor
            else:
                metrica_value = metrica  # Asume que ya es un valor escalar

            # Agregar los resultados a la lista
            results.append({'p': p, 'd': d, 'q': q, 'MAE': metrica_value})

# Crear un DataFrame con los resultados
results_df = pd.DataFrame(results).min()

end_time = time.time() - start_time

print(f'PARA EL MODELO ARIMAX')
print(f'VARIABLES EXOGENAS: {exog_features}')
print(f'Tiempo de ejecución: {end_time:.2f} segundos')
print(f'p = {results_df[0]}, d = {results_df[1]}, q = {results_df[2]}, MAE = {results_df[3]}')

PARA EL MODELO ARIMAX
VARIABLES EXOGENAS: ['diasem', 'trim', 'festivo']
Tiempo de ejecución: 10.93 segundos
p = 1.0, d = 1.0, q = 1.0, MAE = 1135.6786522360646


## Resultados de modelo árbol de decisión

In [26]:
modelo = 'decision_tree'

exog_features = []

lags = 7

def search_space(trial):
    search_space = {
        'max_depth'             : trial.suggest_int('max_depth', 3, 20, step=1),                  # Profundidad máxima del árbol
        'max_leaf_nodes'        : trial.suggest_int('max_leaf_nodes', 30, 70, step=1),            # Máximo número de hojas en el árbol
        'min_samples_leaf'      : trial.suggest_int('min_samples_leaf', 1, 10, step=1),          # Mínimo de muestras por hoja
        'min_samples_split'     : trial.suggest_int('min_samples_split', 2, 20, step=1),         # Mínimo de muestras para dividir un nodo
    }
    return search_space

n_iter = 100

In [27]:
start_time = time.time()

resultado = generic_model(modelo, df, 7, exog_features, search_space, n_iter)

end_time = time.time() - start_time

print(f'PARA EL MODELO {modelo}')
print(f'VARIABLES EXOGENAS: {exog_features}')
print(f'Tiempo de ejecución: {end_time:.2f} segundos')
print(f'{resultado[2]}')
print(f'HIPERPARAMETROS: {resultado[1]}')

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4 5 6 7] 
  Parameters: {'max_depth': 17, 'max_leaf_nodes': 54, 'min_samples_leaf': 8, 'min_samples_split': 2}
  Backtesting metric: 946.2451675634615

PARA EL MODELO decision_tree
VARIABLES EXOGENAS: []
Tiempo de ejecución: 67.81 segundos
mean_absolute_error    946.245168
Name: 0, dtype: float64
HIPERPARAMETROS: {'max_depth': 17, 'max_leaf_nodes': 54, 'min_samples_leaf': 8, 'min_samples_split': 2}


In [37]:
exog_features = ['diasem', 'trim', 'festivo']
maxiter = 50

# Definir los rangos para p, d y q
range_values = range(1, 2) 

In [38]:

# Crear una lista para almacenar los resultados
results = []

# Iterar sobre los valores de p, d, q
for p in range_values:
    for d in range_values:
        for q in range_values:
            # Llamar a la función arimax con los parámetros actuales
            metrica = arimax(exog_features, p, d, q, maxiter)
            
            # Si la métrica es un DataFrame o una Serie, extrae el valor escalar
            if isinstance(metrica, pd.Series):
                metrica_value = metrica.iloc[0]  # Extrae el primer valor
            else:
                metrica_value = metrica  # Asume que ya es un valor escalar

            # Agregar los resultados a la lista
            results.append({'p': p, 'd': d, 'q': q, 'MAE': metrica_value})

# Crear un DataFrame con los resultados
df_results = pd.DataFrame(results)

# Guardar el DataFrame en un archivo Excel
df_results.min()


c:\Users\GregorioValverdeNava\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\GregorioValverdeNava\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


  0%|          | 0/411 [00:00<?, ?it/s]

p         1.000000
d         1.000000
q         1.000000
MAE    1135.678652
dtype: float64

## Resultados de modelo Random Forest

In [16]:
modelo = 'random_forest'

exog_features = []

lags = 7

def search_space(trial):
    search_space = {
        'max_depth'             : trial.suggest_int('max_depth', 3, 20, step=1),                  # Profundidad máxima del árbol
        'max_leaf_nodes'        : trial.suggest_int('max_leaf_nodes', 30, 70, step=1),            # Máximo número de hojas en el árbol
        'min_samples_leaf'      : trial.suggest_int('min_samples_leaf', 1, 10, step=1),          # Mínimo de muestras por hoja
        'min_samples_split'     : trial.suggest_int('min_samples_split', 2, 20, step=1),         # Mínimo de muestras para dividir un nodo
        'max_samples'           : trial.suggest_float('max_samples', 0.5, 1.0)
    }
    return search_space

n_iter = 1

In [17]:
start_time = time.time()

resultado = generic_model(modelo, df, 7, exog_features, search_space, n_iter)

end_time = time.time() - start_time

print(f'PARA EL MODELO {modelo}')
print(f'VARIABLES EXOGENAS: {exog_features}')
print(f'Tiempo de ejecución: {end_time:.2f} segundos')
print(f'{resultado[2]}')
print(f'HIPERPARAMETROS: {resultado[1]}')

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4 5 6 7] 
  Parameters: {'max_depth': 15, 'max_leaf_nodes': 41, 'min_samples_leaf': 3, 'min_samples_split': 12, 'max_samples': 0.8597344848927815}
  Backtesting metric: 902.6027192973853

PARA EL MODELO random_forest
VARIABLES EXOGENAS: []
HIPERPARAMETROS: {'max_depth': 15, 'max_leaf_nodes': 41, 'min_samples_leaf': 3, 'min_samples_split': 12, 'max_samples': 0.8597344848927815}
mean_absolute_error    902.602719
Name: 0, dtype: float64


## Resultados de modelo XGBoost

In [31]:
modelo = 'xgboost'

exog_features = []

lags = 7

def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 200, 1000, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 1),
        'subsample'       : trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'gamma'           : trial.suggest_float('gamma', 0, 1),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1),
    } 
    return search_space

n_iter = 1

In [32]:
start_time = time.time()

resultado = generic_model(modelo, df, 7, exog_features, search_space, n_iter)

end_time = time.time() - start_time

print(f'PARA EL MODELO {modelo}')
print(f'VARIABLES EXOGENAS: {exog_features}')
print(f'Tiempo de ejecución: {end_time:.2f} segundos')
print(f'{resultado[2]}')
print(f'HIPERPARAMETROS: {resultado[1]}')

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4 5 6 7] 
  Parameters: {'n_estimators': 800, 'max_depth': 5, 'learning_rate': 0.2345829390285611, 'subsample': 0.5961832921746021, 'colsample_bytree': 0.7475220728070068, 'gamma': 0.42310646012446096, 'reg_alpha': 0.9807641983846155, 'reg_lambda': 0.6848297385848633}
  Backtesting metric: 956.3783939704987

PARA EL MODELO xgboost
VARIABLES EXOGENAS: []
Tiempo de ejecución: 6.47 segundos
mean_absolute_error    956.378394
Name: 0, dtype: float64
HIPERPARAMETROS: {'n_estimators': 800, 'max_depth': 5, 'learning_rate': 0.2345829390285611, 'subsample': 0.5961832921746021, 'colsample_bytree': 0.7475220728070068, 'gamma': 0.42310646012446096, 'reg_alpha': 0.9807641983846155, 'reg_lambda': 0.6848297385848633}


## REPRESENTACIÓN GRÁFICA DE LOS RESULTADOS

In [ ]:
predictions = resultado[0]

plt.figure(figsize=(10, 5))  # Define el tamaño de la figura
plt.plot(data_test.index, data_test['demanda'], label="Real Value", color='blue', linestyle='-') # Valores reales
plt.plot(predictions.index, predictions['pred'], label="Predictions", color='orange', linestyle='--') # Valores predichos

# Configurar el título y las etiquetas
plt.title("Real Value vs Predicted in Test Data")
plt.xlabel("Date Time")
plt.ylabel("Users")

# Añadir leyenda
plt.legend(loc='upper left')

# Mostrar el gráfico
plt.grid()  # Añadir cuadrícula para mejor visualización
plt.tight_layout()  # Ajustar el layout
plt.show()